In [2]:
pip install selenium

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
Note: you may need to restart the kernel to use updated packages.


ERROR: requests 2.24.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.9 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import requests
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from tqdm import tqdm 
import re

In [3]:
#2
url = 'http://abpat.kipris.or.kr/abpat/searchLogina.do?next=MainSearch'

options = webdriver.ChromeOptions()
# options.add_argument('user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.7.9')
# driver = webdriver.Chrome(executable_path='chromedriver',options=options)

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url=url)
driver.find_element_by_name('queryText').send_keys('CPC=[H]*GD=[20141123~20141131]')   # 검색식 입력
driver.find_element_by_xpath('//*[@id="SearchPara"]/fieldset/span[1]/a/img').click()  # 검색
time.sleep(3)

driver.find_element_by_xpath('//*[@id="liselect_us"]/span[1]').click()   # '미국' 선택
driver.find_element_by_id('divSelSort_1').click()  # 정렬
time.sleep(1)
driver.find_element_by_id('btnSelSortLayer_1_12').click()   # 출원일자 기준 정렬
time.sleep(1)
driver.find_element_by_id('btnSortOrderAsc_1').click()    # 오름차순
time.sleep(1)
driver.find_element_by_xpath('//*[@id="leftside"]/fieldset[2]/span').click()    # 정렬완료

select = Select(driver.find_element_by_id("opt28"))  # 90개씩 보기 설정
select.select_by_index(2)    

driver.find_element_by_xpath('//*[@id="detail_content"]/div[1]/div[4]/a').click()  # 검색
time.sleep(2)

page_num = driver.find_element_by_css_selector('#divMainArticle > form > section > div.articles_page > p').text  # 전체 페이지수
page_num = page_num.split()[-2].replace(',','')
page_num = int(page_num)

pat_num =  driver.find_element_by_css_selector('#divMainArticle > form > section > div.articles_page > p.articles > span.total').text
pat_num = int(pat_num.replace(',',''))

total = []
for page in tqdm(range(1,page_num+1)):
    pg = page % 10
    if page == page_num:
        for num in range(1,(pat_num % 90)+1):
            contents = [] # 특허 정보 담을 리스트

            # 제목
            title = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num}]/div[1]/h1/a').text   
            # CPC
            try:
                cpc = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num}]/div[2]/ul/li[2]/span[2]/font').get_attribute("title")
                cpc = cpc.split(' ')
            except NoSuchElementException:
                cpc = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num}]/div[2]/ul/li[2]').text
                cpc = cpc.split()[2:]

            contents.append(title)
            contents.append(cpc)
            
            # 출원번호, 출원일자, 등록번호, 등록일자, 공개번호, 공개일자, 공보번호, 공보일자
            for i in range(3,11):
                try:
                    info = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num}]/div[2]/ul/li[{i}]').text
                except StaleElementReferenceException:
                    time.sleep(2)
                    info = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num}]/div[2]/ul/li[{i}]').text
                
                contents.append(info.split(' :')[-1])

            total.append(contents)    

    else:
        for num in range(90):
            contents = [] # 특허 정보 담을 리스트

            # 제목
            title = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num+1}]/div[1]/h1/a').text   
            # CPC
            try:
                cpc = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num+1}]/div[2]/ul/li[2]/span[2]/font').get_attribute("title")
                cpc = cpc.split(' ')
            except NoSuchElementException:
                cpc = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num+1}]/div[2]/ul/li[2]').text
                cpc = cpc.split()[2:]

            contents.append(title)
            contents.append(cpc)
            
            
            # 출원번호, 출원일자, 등록번호, 등록일자, 공개번호, 공개일자, 공보번호, 공보일자
            for i in range(3,11):
                try:
                    info = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num+1}]/div[2]/ul/li[{i}]').text
                except StaleElementReferenceException:
                    time.sleep(2)
                    info = driver.find_element_by_xpath(f'/html/body/div[2]/div/section[3]/section/div[3]/form/section/article[{num+1}]/div[2]/ul/li[{i}]').text
          
                contents.append(info.split(' :')[-1])

            total.append(contents)    

        if pg != 0:
            driver.find_element_by_css_selector(f'#divBoardPager > a:nth-child({pg+2})').click()
            time.sleep(2)
        else:
            driver.find_element_by_css_selector('#divBoardPager > a.next').click()
            time.sleep(2)

  0%|                                                                                           | 0/23 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'split'

In [6]:
df = pd.DataFrame(total,columns=['제목','CPC','출원번호','출원일자','등록번호','등록일자','공개번호','공개일자','공보번호','공보일자'])
df

,제목,CPC,출원번호,출원일자,등록번호,등록일자,공개번호,공개일자,공보번호,공보일자
0,CATHODE FOR MOLTEN CARBONATE FUEL CELLS HAVING...,"[H01M8/142, H01M4/8647, H01M4/9025, H01M8/145,...",14551064,2014.11.23,,,20150303507,2015.10.22,20150303507,2015.10.22
1,Apparatus and method for calculating regenerat...,"[B60L7/18, B60T8/326, B60T2270/604, B60W10/08,...",14551025,2014.11.23,09561726,2017.02.07,20160031325,2016.02.04,09561726,2017.02.07
2,Regenerative brake apparatus of hybrid vehicle...,"[B60T7/18, B60T8/1701, B60T2270/604, Y02T10/64...",14551021,2014.11.23,09371061,2016.06.21,20160059705,2016.03.03,09371061,2016.06.21
3,Spacing gauge with thermal indicia for solar p...,"[F24S25/636, G01B5/14, F24S25/63, F24S25/632, ...",14551058,2014.11.23,08991065,2015.03.31,,,08991065,2015.03.31
4,Rail Splice with Thermal Indicia for Solar Pan...,"[F24S25/636, F24J2/5245, F24J2/5252, F24J2/525...",14551056,2014.11.23,,,20150078806,2015.03.19,20150078806,2015.03.19
...,...,...,...,...,...,...,...,...,...,...
944,Super capacitor structure and the manufacture ...,"[H01G11/32, H01G11/36, H01G11/38, H01G11/40, H...",14556229,2014.11.30,09208959,2015.12.08,20150121671,2015.05.07,09208959,2015.12.08
945,Water faucet with integrated contact and conta...,"[C02F1/725, C02F2209/44, C02F2303/04, C02F2305...",14556175,2014.11.30,,,20150158750,2015.06.11,20150158750,2015.06.11
946,System and method for diagnosing state of fuel...,"[H01M8/04492, G01R31/3658, H01M2250/20, Y02T90...",14556179,2014.11.30,09929418,2018.03.27,20150362560,2015.12.17,09929418,2018.03.27
947,APPARATUS AND METHOD FOR DIAGNOSING DEGRADATIO...,"[G01R31/392, G01R31/3679, H01M10/48, G01R31/36...",14556182,2014.11.30,,,20160084917,2016.03.24,20160084917,2016.03.24


In [7]:
df.to_excel('201411-3.xlsx')